<b>Проект:</b> Изучение и анализ данных о пользователях сайта Википедия и о проводимых на нем выборах.


<b>Цель проекта:</b> Изучить имеющиеся данные, привести их в пригодный для анализа формат.
Обнаружить в них значимые закономерности. 
Выбрать параметры для сегментации, разделить пользователей на сегменты и изучить данные подробнее в разрезе сегментов. 
Исследовать голосующую аудиторию проекта, динамику голосований. 


<b>Описание данных:</b> Имеются 2 файла данных за 2008-2023гг. о пользователях сайта Википедия (stats и votes).
В файле stats содержатся данные о ТОП-150 пользователей, который определяется по количеству правок, сделанных пользователем.


# Изучение и предобработка данных.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
import math

Загрузим данные.

In [4]:
stats.head()

,Edits,Reverts,Log,Diff,Volume,Tot size,Time,Speed,User,txt
0,1589,NaN,1621.0,262 k,1.1 M,39 M,95 h,34/h,Altes,200708
1,1809,NaN,1328.0,395 k,2.8 M,42 M,129 h,24/h,Obersachse,200708
2,2734,NaN,NaN,16 k,15 k,19 M,105 h,26/h,Monegasque~ruwiki,200708
3,2611,NaN,6.0,278 k,350 k,43 M,135 h,19/h,User№101,200708
4,917,6.0,1416.0,154 k,1.2 M,26 M,98 h,24/h,Kalan,200708


In [5]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28950 entries, 0 to 28949
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Edits     28950 non-null  int64  
 1   Reverts   10157 non-null  float64
 2   Log       28306 non-null  float64
 3   Diff      28945 non-null  object 
 4   Volume    28945 non-null  object 
 5   Tot size  28945 non-null  object 
 6   Time      28950 non-null  object 
 7   Speed     28950 non-null  object 
 8   User      28950 non-null  object 
 9   txt       28950 non-null  int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 2.2+ MB


Проведем предобработку данных:
Приведем названия столбцов к нижнему и змеиному регистру.
Столбец 'txt' переведем в формат datetime.

In [6]:
stats.columns = stats.columns.str.lower()

In [7]:
stats.rename(columns={'tot size':'tot_size'}, inplace=True)

In [8]:
stats['txt'] = pd.to_datetime(stats['txt'],format='%Y%m')

Напишем функцию для перевода данных в столбцах diff, volume, tot_size в формат float с единицами измерения в тысячах единиц.

In [9]:
def measure_i(measure):
         
    if "k" in str(measure):
        measure = pd.to_numeric(str(measure).split()[0],downcast='signed',errors='coerce')
        return measure
    elif "M" in str(measure):
        measure = pd.to_numeric(str(measure).split()[0],downcast='signed',errors='coerce')
        return measure*1000
    else:
        return pd.to_numeric(measure,errors='coerce')/1000   


In [ ]:
cols_to_change = ['diff', 'volume', 'tot_size']

for col in cols_to_change:
    stats[col] = stats[col].apply(measure_i)

Преобразуем данные в столбцах speed, time в числовой формат с единицами измерения колличество правок в минуту и минуты. Переименуем данные столбцы.

In [ ]:
stats['speed']=pd.to_numeric(stats.speed.str.split('/', expand=True)[0],errors='coerce')/60

In [ ]:
stats.rename(columns={'speed':'speed_ed_min'}, inplace=True)

In [ ]:
stats['time'] = pd.to_numeric(stats.time.str.split('\xa0', expand=True)[0],errors='coerce')*60+pd.to_numeric(stats.time.str.split('\xa0', expand=True)[2],errors='coerce').fillna(0)

In [ ]:
stats.rename(columns={'speed':'speed_ed_min','time':'time_min'}, inplace=True)

Заполним пропуски в столбцах revert, log нулями, руководствуясь соображениями, что значения пропущены у тех пользователей, кто не отменял чужие правки и не совершал иных действий.

In [ ]:
stats['reverts']=stats['reverts'].fillna(0)

In [ ]:
stats['log']=stats['log'].fillna(0)

In [ ]:
stats.info()

Проверим данные на явные дубликаты.

In [ ]:
stats.duplicated().sum()

Узнаем количество уникальных пользователей.

In [ ]:
user_count = stats['user'].nunique()
user_count

В ячейках diff,volume,tot_size по 5 нулевых значений. Выгрузим строки с нулевыми значениями по столбцу diff.

In [ ]:
stats.loc[stats['volume'].isnull()]

Видим, что именно в этих строках отсутствуют значения по 2 другим столбцах. Т.к. их количество всего 5, удалим эти строки, т.к. это не повлияет на результаты анализа.

In [ ]:
stats=stats.dropna(subset=['volume'])

In [ ]:
stats.head()

# Исследовательский анализ данных.

Т.к. статистика каждый месяц формируется только по Топ-150 пользователей по количеству правок, вся аналитика будет вестись с учетом этой особенности. 

Для начала проанализируем общее изменение показателей по годам по средним показателям и по общим. Показатели 'volume','diff','tot_size' отражены в тысячах, а 'edits','log','reverts' - в единицах.

In [ ]:
stats.groupby(['txt']).agg({'volume':['mean'],'diff':['mean']}).plot(figsize=(10,5),grid=True)
plt.title('Изменение средних показателей добавленного объема и добавленного объема за вычетом удаленного по годам')
plt.xlabel('Год')
plt.ylabel('Объем добавленного,тыс.ед.')
plt.show()

In [ ]:
stats.groupby(['txt']).agg({'edits':['mean'],'log':['mean'],'reverts':['mean']}).plot(figsize=(10,5),grid=True)
plt.title('Изменение средних показателей количества правок, иных действий и отмен чужих правок по годам')
plt.xlabel('Год')
plt.ylabel('Количество действий')
plt.show()

In [ ]:
stats.groupby(['txt']).agg({'volume':['sum'],'diff':['sum']}).plot(figsize=(10,5),grid=True)
plt.title('Изменение суммарных показателей добавленного объема и добавленного объема за вычетом удаленного по годам')
plt.xlabel('Год')
plt.ylabel('Объем добавленного,тыс.ед.')
plt.show()

In [ ]:
stats.groupby(['txt']).agg({'edits':['sum'],'log':['sum'],'reverts':['sum']}).plot(figsize=(10,5),grid=True)
plt.title('Изменение суммарных показателей количества правок, иных действий и отмен чужих правок по годам')
plt.xlabel('Год')
plt.ylabel('Количество действий')
plt.show()

Из графиков видно, что все показатели имеют достаточно стабильный уровень, начиная с 2009года.  
Отмечается пик роста количества правок в 2015-2016гг.(особенно в середине 2015г.-более 1 750 в среднем и более 1 750 - в общем), а снижение в 2008,2009,2014,2018,2020гг.  
Общий объем написанного(и объщий объем за вычетом удаленного-diff) имеет пик в конце 2017г(около 900 000(700 000) в среднем и более 140 000 000(110 000) в общем).   
А активность иных действий максимальная в середине 2011г.

In [ ]:
stats.groupby(['txt']).agg({'tot_size':['mean']}).plot(figsize=(10,5),grid=True)
plt.title('Изменение средней накопленной суммы добавленного по годам')
plt.xlabel('Год')
plt.ylabel('Накопленная сумма добавленного,тыс.ед.')
plt.show()

In [ ]:
stats.groupby(['txt']).agg({'tot_size':['sum']}).plot(figsize=(10,5),grid=True,legend=False)
plt.title('Изменение суммарной накопленной суммы добавленного по годам')
plt.xlabel('Год')
plt.ylabel('Накопленная сумма добавленного,тыс.ед.')
plt.show()

Пики роста по средней и суммарной накопленной сумме добавленного наблюдаются в 2013,2016,2021,2023гг.

In [ ]:
stats.groupby(['txt']).agg({'speed_ed_min':['mean']}).plot(figsize=(10,5),grid=True,legend=False)
plt.title('Средняя скорость внесения правок по годам')
plt.xlabel('Год')
plt.ylabel('Средняя скорость,ед/мин')
plt.show()

Средняя скорость внесения правок имеет тенденцию к росту с 2019, что может быть связано с использованием ботов при написании статей.

Далее сгруппируем таблицу по пользователям. 
Сегметируем пользователей по частоте попадания в ТОП-150 и рассмотрим, как меняются показатели в зависимости от группы. 
Сгруппируем данные по пользователям, применив к 'volume','diff','edits','tot_size','log','reverts' функцию среднего,посчитаем количество раз попадания пользователя, время первого и последнего попадания в ТОП.
Переименуем столбцы, а также найдем периодичность в месяцах, с какой каждый пользователь попадал в ТОП-150.

In [ ]:
frec_group = stats.groupby(['user']).agg({'txt':['min','max','count'],'volume':['mean'],'diff':['mean'],'tot_size':['mean'],'edits':['mean'],'log':['mean'],'reverts':['mean']})


In [ ]:
frec_group.index.names = ['voter']

In [ ]:
frec_group.columns = ['txt_min','txt_max','count_in_top','volume','diff','tot_size','edits','log','reverts']

In [ ]:
frec_group['txt_delta_d'] = frec_group['txt_max']-frec_group['txt_min']

In [ ]:
frec_group['txt_delta_d'] = frec_group['txt_delta_d'].dt.days.astype('int')

Рассчитаем в месяцах период, в течение которого пользователь периодически попадал в ТОП-150.

In [ ]:
frec_group['txt_delta_m'] = round(frec_group['txt_delta_d']/30.44)


Добавим столбец с расчетом частоты попадания пользователя в ТОП в месяцах.

In [ ]:
frec_group['frec_top_m'] = round(frec_group['txt_delta_m']/frec_group['count_in_top'],1)

In [ ]:
frec_group.sort_values(by=['count_in_top'],ascending=False)

Построим гистограмму, чтобы определить, распределение количества раз попадания в ТОП-150. Перед построением исключим пользователей, которые попали в Топ только 1 раз, т.к. их можно отнести к группе самых редко попадающих в Топ пользователей.

In [ ]:
frec_group.plot(y='count_in_top',kind='hist',bins=100,figsize=(10,5),grid=True,legend=False)
plt.title('Распределение количества раз попадания пользователем в ТОП-150')
plt.xlabel('Количество попаданий в ТОП-150')
plt.ylabel('Количество пользователей')
plt.show()

Видим, что большинство пользователей попадало в ТОП только 1-2 раза. Достаточно много от 3 до 10, затем показатель сильно снижается. Напишем функцию для разбивки пользователей на группы по количеству раз попадания в ТОП-150 и применим ее для создания нового столбца с указанием номера группы каждого пользователя.

Разобьем пользователей на сегменты по количеству раз попадания в ТОП: 
1- 1 раз , 
2 - от 1 до 5 раз, 
3 - от 6 до 10 раз, 
4 - от 11 до 25 раз, 
5 - от 26 до 50 раз, 
6 - от 51 до 75 раз,
7 - более 76 раз.

In [ ]:
def frequency(interval):
         
    if 0 < interval <= 1:
        return 1
    elif 1 < interval <= 5:
        return 2
    elif 5 < interval <= 10:
        return 3
    elif 10 < interval <= 25:
        return 4
    elif 25 < interval <= 50:
        return 5
    elif 50 < interval <= 75:
        return 6
    else:
        return 7

In [ ]:
frec_group['group_in_top'] = frec_group['count_in_top'].apply(frequency)

Сгруппируем таблицу по номерам групп, применив 'volume','diff','edits','log' функцию среднего и посчитаем количество пользователей в каждой группе. Затем построим графики по этим данным.

In [ ]:
frec_group_mean_1 = frec_group.groupby(['group_in_top']).agg({'group_in_top':['count'],'volume':['mean'],'diff':['mean'],'edits':['mean'],'log':['mean']})

In [ ]:
frec_group_mean_1

In [ ]:
frec_group_mean_1.plot(figsize=(10, 5))
plt.title('Средние показатели групп')
plt.xlabel('Номер группы')
plt.ylabel('Средний объем/Среднее количество')
plt.show()

Пользователей 1 и 2 группы больше всего(попадали в ТОП от 1 до 5 раз), потом этот показатель резко снижается и падает с возрастанием количества раз попадания в ТОП.
По средним показателям 'volume','diff','edits','log' наблюдается динамика роста с ростом количества раз попадания пользователя в ТОП.
В 5 группе(25-50 попаданий в ТОП) наблюдается повышение количества правок и иных действий при одновременном сокращении объема написанного и написанного за вычетом удаленного.

In [ ]:
frec_group_mean_1.columns = ['count','vol_mean','diff_mean','edits_mean','log_mean']

Посчитаем долю пользователей каждой группы относительно общего объема.

In [ ]:
frec_group_mean_1['share']=round(frec_group_mean_1['count']/user_count*100,1)

In [ ]:
frec_group_mean_1

In [ ]:
frec_group_mean_1.plot(kind='pie',y='count',autopct='%.1f%%',legend=False,title='Доля пользователей каждой группы относительно общего объема');

Примерно одинаковое количество представителей 1 группы(1 раз попадание) -30,1% и 2 группы(2-5 раз) - 31,3%; более, чем в 2 раза меньше представителей 3 и 4 группы(5-10 и 11-25 раз) - 14,1% и 11,8%, соответственно. Те, кто оказался в ТОПе более 50раз - 5,6%

In [ ]:
frec_group.groupby(['group_in_top']).agg({'reverts':['mean']}).plot(figsize=(7, 3),legend=False)
plt.title('Среднее количество отмен чужих правок по группам')
plt.xlabel('Номер группы')
plt.ylabel('Количество отмен')
plt.show()

Также наблюдаем динамику, что пользователи, которые чаще попадают в ТОП, делают больше отмен чужих правок. Возможно это связано с большим объемом и количеством написанного, соответственно в большее количество текстом могли внести правки другие пользователи, с которыми автор мог не согласиться.

In [ ]:
frec_group.groupby(['group_in_top']).agg({'tot_size':['mean']}).plot(figsize=(7, 3),legend=False)
plt.title('Общая накопленная сумма добавленного по группам')
plt.xlabel('Номер группы')
plt.ylabel('Накопленная сумма добавленного')
plt.show()

Действительно график изменения общего объема накопленного по группам очень похож на график отмены чужих правок, что подтверждает предположение. Общий объем накопленного расчет в зависимости от количества раз попадания пользователя в ТОП. Также стоит отметить, что в последней группе находятся самые "старые" авторы проекта. 

In [ ]:
frec_group.head(5)

Напишем функцию для класиффикации пользователей по частоте попадания в ТОП в месяцах(т.е. раз во сколько месяцев в среднем пользователь попадает в ТОП), и применим эту классификацию внутри предыдущей классификации.
Разобьем пользователей на сегменты по частоте попадания в ТОП: 
1- раз в 1 месяц, 
2 - раз в 1 - 2 мес., 
3- раз в 2 - 3 мес., 
4 - раз в 3 - 6мес., 
5 - раз в 6 - 12 мес. 
6- реже 1 раза в год либо попавшие 1 раз в ТОП.

In [ ]:
def frequency_2(interval_2):
         
    if 0 < interval_2 <= 1:
        return 1
    elif 1 < interval_2 <= 2:
        return 2
    elif 2 < interval_2 <= 3:
        return 3
    elif 3 < interval_2 <= 6:
        return 4
    elif 6 < interval_2 <= 12:
        return 5
    else:
        return 6

In [ ]:
frec_group['frec_int'] = frec_group['frec_top_m'].apply(frequency_2)

In [ ]:
frec_group

In [ ]:
frec_group.query('frec_top_m!=0').plot(y='frec_top_m',kind='hist',bins=100,figsize=(10,5),grid=True)
plt.title('Распределение частоты попадания пользователем в ТОП-150')
plt.xlabel('Частота попаданий в ТОП-150, мес.')
plt.ylabel('Количество пользователей')
plt.show()

Получается, что в ТОПе в основном присутствуют пользователи, которые попадают в ТОП не реже 1раза в 1-3 месяца.

Далее проанализируем частоту попадания внутри групп, выделенных по общему количеству попадания в ТОП.

In [ ]:
frec_group.query('group_in_top == 2').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 2');

In [ ]:
frec_group.query('group_in_top == 2').groupby(['frec_int']).agg({'frec_int':['count']}).plot(kind='bar',legend=False)
plt.title('Распределение частоты попадания пользователем в ТОП-150 для 2 группы')
plt.xlabel('Частота попаданий в ТОП-150, мес.')
plt.ylabel('Количество пользователей')
plt.show()

В группе, кто попал в ТОП 2-5 раз почти половина пользователей попадала в него с периодичностью раз в месяц, т.е. им удавалось 2-5 месяцев подряд поддерживать высокий уровень правок и оставаться в ТОПе. Остальная часть примерно одинаково распределена по группам.

In [ ]:
frec_group.query('group_in_top == 3').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 3');

В группе, кто попал в ТОП 6-10 раз примерно 1/3 пользователей попадала в него с периодичностью 1-2 раза в месяц. 

In [ ]:
frec_group.query('group_in_top == 4').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 4');

В группе, кто попал в ТОП 11-25 раз, у большей части периодичностью попадания либо 1-2 раза в месяц или раз в 3.1-6 месяцев. 

In [ ]:
frec_group.query('group_in_top == 5').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 5');

В группе, кто попал в ТОП 26-50 раз, периодичностью попадания в основном либо раз в 1-2 месяца, либо раз в 3.1-6 месяцев, либо раз в 6.1-12 месяцев. 

In [ ]:
frec_group.query('group_in_top == 6').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 6');

Кто попадал в ТОП 51-75 раз, почти половина пользователей попадала туда раз в 1-2 месяца. 

In [ ]:
frec_group.query('group_in_top == 7').groupby(['frec_int']).agg({'frec_int':['count']})\
.plot(kind='pie',subplots=True,autopct='%.1f%%',legend=False,title='Доля пользователей по частоте попадания в ТОП для группы 7');

А подавляющее большинство тех, кто попал в ТОП более 75 раз, оказывался там с периодичностью раз в 1-2 месяца.


Т.о., большинство пользователей, которые оказались в ТОПе более 50 раз, имеют стабильно высокий уровень правок, который позволяет им попадать в ТОП раз в 1-2 месяца. Так же, как и у новичков, которые побывали в ТОПе 2-5 раз.

Проведем предобработку данных в таблице votes.

In [ ]:
votes

Переведем столбец 'time' к формату datetime64

In [ ]:
votes['time'] = pd.to_datetime(votes['time'])

In [ ]:
votes.info()

Добавим столбец, который обозначает день голосования в целых числах.

In [ ]:
votes['lt_day']= votes['lt'].apply(np.ceil)

Исключим из выборки тех пользователей, которые не имеют право голоса.

In [ ]:
votes =votes.query('can_vote == True')

In [ ]:
votes['lt'].sort_values()

В значениях суток от начала голосования есть нулевые значения, скорее всего связанные с тем, что люди проголосовали до начала голосований раньше времени. Исключим данные по данным пользователям.

In [ ]:
votes =votes.query('lt != 0')

Сгруппируем данные по дню голосования и найдем количество голосов, отданных в этот день и определим, на сколько % количество голосов "за" превышает количество голосов "против". Построим графики изменения этих показателей в зависимости от дня голосования.

In [ ]:
votes_day=votes.groupby(['lt_day']).agg({'vote':['count','mean']})

In [ ]:
votes_day.columns = ['count','mean']

In [ ]:
votes_day

In [ ]:
plt.figure(figsize=(6,3))
votes_day['count'].plot()
plt.title('Количество голосов по дням голосования')
plt.xlabel('День голосования')
plt.ylabel('Количество голосов')
plt.show()

In [ ]:
plt.figure(figsize=(6,3))
votes_day['mean'].plot()
plt.title('Доля голосов "за" по дням голосования')
plt.xlabel('День голосования')
plt.ylabel('Доля голосов "за"')
plt.show()

Максимальное количество голосов отдается в 1 день голосования, потом количество резко падает и продолжает снижаться до 5  дня. После 5 дня начинается небольшой рост. Возможно это связано с тем, что определяются основные претенденты и начинают отдавать голоса, чтобы поддержать их.

Это предположение можно подтвердить и тем, что с 5 дня начинают больше голосовать "за", чем "против".

Посчитаем, сколько голосов отдал каждый пользователь и в скольки голосованиях он участвовал. Затем объединим данные с талицей stats и посчитаем долю голосующих, которые попадают в ТОП-150 по количеству правок.

In [ ]:
votes_group=votes.groupby(['voter']).agg({'n':['count','nunique']})

In [ ]:
votes_group.columns = ['vote_cnt','n_cnt']

In [ ]:
votes_group.index.names = ['voter']

In [ ]:
votes_group['vote_cnt_n'] = round(votes_group['vote_cnt']/votes_group['n_cnt'],1)

In [ ]:
votes_group

In [ ]:
votes_group_n=votes_group.join(frec_group, on='voter').fillna(0)

In [ ]:
votes_group_n

In [ ]:
round(votes_group_n.query('count_in_top != 0')['vote_cnt'].count()/votes_group_n['vote_cnt'].count()*100,2)

Доля голосующих, которые оказываются в ТОП-150 по правках, составляет 59,8%. Получается, что из числа, кто активно делает правки статей, голосуют лишь чуть больше половины пользователей.

Посчитаем в какой группе (в зависимости от количества раз попадания в ТОП за период 2008-2023гг) люди чаще голосуют и в какой отдают больше всего голосов за одно голосование. Возьмем средние значения по группе.

In [ ]:
top_count_vote = round(votes_group_n.query('count_in_top != 0').groupby('group_in_top')\
                       .agg({'vote_cnt':['mean'],'vote_cnt_n':['mean'],'n_cnt':['mean']}),2)

In [ ]:
top_count_vote.columns=['vote_cnt','vote_cnt_n','n_cnt']

In [ ]:
top_count_vote['vote_cnt'].plot(figsize=(5,3),kind='bar')
plt.title('Среднее количество голосов по группам попадания в ТОП-150')
plt.xlabel('Группа попадания в ТОП-150')
plt.ylabel('Количество голосов') 
plt.show()

In [ ]:
top_count_vote['vote_cnt_n'].plot(figsize=(5,3),kind='bar')
plt.title('Среднее количество голосов, отданных 1 пользователем за 1 голосование, по группам попадания в ТОП-150')
plt.xlabel('Группа попадания в ТОП-150')
plt.ylabel('Количество голосов') 
plt.show();

In [ ]:
top_count_vote['n_cnt'].plot(figsize=(5,3),kind='bar')
plt.title('Среднее количество голосований, в которых участвовал пользователь, по группам попадания в ТОП-150')
plt.xlabel('Группа попадания в ТОП-150')
plt.ylabel('Количество голосований') 
plt.show()

Получается, что чем больше раз пользователь попадает в ТОП, тем в большем количестве голосований он участвует и отдает больше голосов, т.е. делает это системно. 
Больше всего голосов во время одного голосования отдают те, кто попадал в ТОП от 5 до 10 раз, а меньше всего те, кто попадал в ТОП более 75 раз. 

In [ ]:
votes

In [ ]:
votes['n_str']=votes['n'].astype('str')

In [ ]:
votes['candidate_n'] = votes['candidate'] + '_' + votes['n'].astype('str')

In [ ]:
votes['candidate_n'] = votes['candidate_n'].str.replace('.','_',regex=False)

Отберем только тех пользователей, которые имеют право голосовать.

In [ ]:
votes = votes.query('can_vote == True')

In [ ]:
votes

Сгруппируем данные по кандитатам, номеру голосования и дню голосования. После этого посчитаем количество полученных голосов и долю голосов "за". 

In [ ]:
votes_candidate = votes.groupby(['candidate','n','lt_day'],as_index = False).agg({'vote':['count','mean']})

In [ ]:
votes_candidate.columns = ['candidate','n','lt_day','count_vote','mean_vote']

In [ ]:
votes_candidate

Определим кандидатов, которые достигли необходимого порога в 2/3 голосов 'за' при общем количестве голосов более 20. 

In [ ]:
votes_candidate_1 = votes.groupby(['candidate','n'],as_index = False).agg({'vote':['count','mean']})

In [ ]:
votes_candidate_1.columns = ['candidate','n','count_vote','mean_vote']

In [ ]:
votes_candidate_1['true'] = votes_candidate_1['mean_vote'] > 1/3

In [ ]:
votes_candidate_1

In [ ]:
votes_candidate_2 = votes_candidate_1.query('true == True and count_vote >= 20').groupby(['candidate'],as_index = False).agg({'n':['count']})

In [ ]:
votes_candidate_2.columns = ['candidate','n_count']

In [ ]:
votes_candidate_2.groupby('n_count').agg({'candidate':['count']})

37 кандидатов достигли 1 раз необходимого порога 2/3 голосов "за" за время участия в выборах, 24 - 2 раза, 9 - 3раза, 3 - 4 раза 1 - 6 раз.

In [ ]:
votes_candidate.query('mean_vote > 1/3').groupby(['n']).agg({'candidate':['nunique']}).plot(figsize=(10,3),kind='bar',legend=False)
plt.title('Количество претендентов, которые достигали необходимо порога 2/3 голосов "за", по каждому голосованию')
plt.xlabel('Номер голосования')
plt.ylabel('Количество претендентов') 
plt.show()

Построен график, отображающий количество претендентов, которые достигали необходимо порога 2/3 голосов "за", по каждому голосованию (больше всего их было на 9, 12 , 32 голосовании).


# ВЫВОДЫ

Целью проекта было: 
- изучить имеющиеся данные, привести их в пригодный для анализа формат,  
- обнаружить в них значимые закономерности,  
- выбрать параметры для сегментации, разделить пользователей на сегменты и изучить данные подробнее в разрезе сегментов,  
- исследовать голосующую аудиторию проекта, динамику голосований,  
На основе данных за 2008-2023гг. о пользователях сайта Википедия.  
В файле stats содержались данные о ТОП-150 пользователей, который определяется по количеству правок, сделанных пользователем. В файле votes — информация о голосованиях.  
Была проведена предобработка данных. Названия столбцов приведены к нижнему и змеиному регистру. Столбец с датой переведем в формат datetime.
Данные, которые должны содержать числовые значения переведены в числовой формат.  
Заполнена пропуски в столбцах.  
Данные проверены на явные дубликаты. Удалены 5 строк, в которых отсутствуют большинство необходимых значений.  
Количество уникальных пользователей в таблице stats после предобработки – составило 2 541.  
Проанализировано общее изменение показателей по годам по средним показателям и по общим.  
Из построенных графиков видно, что все показатели имеют достаточно стабильный уровень, начиная с 2009года. Отмечается пик роста количества правок в 2015-2016гг.(особенно в середине 2015г.-более 1 750 в среднем и более 1 750 - в общем), а снижение в 2008,2009,2014,2018,2020гг. Общий объем написанного(и общий объем за вычетом удаленного-diff) имеет пик в конце 2017г(около 900 000(700 000) в среднем и более 140 000 000(110 000) в общем). А активность иных действий максимальная в середине 2011г.  
Пики роста по общему объему и количеству правок наблюдаются в 2012,2015,2021,2023гг.  
Средняя скорость внесения правок имеет тенденцию к росту с 2019, что может быть связано с использованием ботов при написании статей.  
Далее данные сгруппированы по пользователям, к 'volume','diff','edits','tot_size','log','reverts' применена функция среднего, посчитано количество раз попадания пользователя в ТОП, а также найдена периодичность в месяцах, с какой каждый пользователь попадал в ТОП-150.  
По гистограмме (в которой исключены пользователи, которые попали в Топ только 1 раз, т.к. их можно отнести к группе самых редко попадающих в Топ пользователей) видно, что большинство пользователей попадало в ТОП только 1-2 раза. Достаточно много от 3 до 10, затем показатель сильно снижается. Напишем функцию для разбивки пользователей на группы по количеству раз попадания в ТОП-150 и применим ее для создания нового столбца с указанием номера группы каждого пользователя.  
Далее проведена сегментация пользователей на 7 групп по частоте попадания в ТОП-150:  
1-	1 раз ,   
2-	 от 1 до 5 раз,   
3-	 от 6 до 10 раз,   
4-	от 11 до 25 раз,   
5-	 от 26 до 50 раз,   
6-	от 51 до 75 раз,   
7-	более 76 раз.  
К столбцам 'volume','diff','edits','log' применена функцию, посчитано количество пользователей в каждой группе.   
Пользователей 1 и 2 группы больше всего, потом этот показатель резко снижается и падает с возрастанием количества раз попадания в ТОП. По средним показателям 'volume','diff','edits','log' наблюдается динамика роста с ростом количества раз попадания пользователя в ТОП. В 5 группе наблюдается повышение количества правок и иных действий при одновременном сокращении объема написанного и написанного за вычетом удаленного.  
Посчитана доля пользователей каждой группы относительно общего объема.  
Примерно одинаковое количество представителей 1 группы -30,1% и 2 группы - 31,3%; более, чем в 2 раза меньше представителей 3 и 4 группы - 14,1% и 11,8%, соответственно. Те, кто оказался в ТОПе более 50 раз - 5,6%  
Также обнаружена динамика, что пользователи, которые чаще попадают в ТОП, делают больше отмен чужих правок. Возможно это связано с большим объемом и количеством написанного, соответственно в большее количество текстом могли внести правки другие пользователи, с которыми автор мог не согласиться. Действительно график изменения общего, объема накопленного по группам очень похож на график отмены чужих правок, что подтверждает предположение. 
Общий объем накопленного растет в зависимости от количества раз попадания пользователя в ТОП. Также стоит отметить, что в последней группе находятся самые "старые" авторы проекта.  
Далее пользователи были сегментированы по частоте попадания в ТОП в месяцах (т.е. раз во сколько месяцев в среднем пользователь попадает в ТОП):  
1-	раз в 1 месяц,   
2-	раз в 1 - 2 мес.,   
3-	раз в 2 - 3 мес.,   
4-	раз в 3 – 6 мес.,   
5-	раз в 6 - 12 мес.,  
6-	реже 1 года либо попавшие 1 раз в ТОП.  

Эта классификация была применена внутри предыдущей классификации (по количеству раз попадания в ТОП). 
Было выявлено, что в ТОПе в основном присутствуют пользователи, которые попадают в ТОП не реже 1 раза в 1-3 месяца.  
В группе, кто попал в ТОП 2-5 раз почти половина пользователей попадала в него с периодичностью раз в месяц, т.е. им удавалось 2-5 месяцев подряд поддерживать высокий уровень правок и оставаться в ТОПе. Остальная часть примерно одинаково распределена по группам.  
В группе, кто попал в ТОП 6-10 раз примерно 1/3 пользователей попадала в него с периодичностью 1-2 раза в месяц.  
В группе, кто попал в ТОП 11-25 раз, у большей части периодичностью попадания либо 1-2 раза в месяц или раз в 3.1-6 месяцев.  
В группе, кто попал в ТОП 26-50 раз, периодичностью попадания в основном либо раз в 1-2 месяца, либо раз в 3.1-6 месяцев, либо раз в 6.1-12 месяцев.
Кто попадал в ТОП 51-75 раз, почти половина пользователей попадала туда раз в 1-2 месяца.  
А подавляющее большинство тех, кто попал в ТОП более 75 раз, оказывался там с периодичностью раз в 1-2 месяца.  
Т.о., большинство пользователей, которые оказались в ТОПе более 50 раз, имеют стабильно высокий уровень правок, который позволяет им попадать в ТОП раз в 1-2 месяца. Так же, как и у новичков ТОПа, которые побывали в ТОПе 2-5 раз.  

Далее анализ велся по таблице votes. Были исключены из выборки те пользователи, которые не имеют право голоса.  
В значениях суток от начала голосования были обнаружены нулевые значения, скорее всего связанные с тем, что люди проголосовали до начала голосований раньше времени. Данные по данным пользователям были исключены.  
Данные были сгруппированы по дню голосования и найдено количество голосов, отданных в этот день и определено, на сколько % количество голосов "за" превышает количество голосов "против". Построены графики изменения этих показателей в зависимости от дня голосования.  
Максимальное количество голосов отдается в 1 день голосования, потом количество резко падает и продолжает снижаться до 5 дня. После 5 дня начинается небольшой рост. Возможно это связано с тем, что определяются основные претенденты и начинают отдавать голоса, чтобы поддержать их. Это предположение можно подтвердить и тем, что с 5 дня начинают больше голосовать "за", чем "против".  

Посчитано, сколько голосов отдал каждый пользователь и в каком количестве голосований он участвовал. Затем данные были объединены с талицей stats и посчитана доля голосующих, которые попадают в ТОП-150 по количеству правок.  
Доля голосующих, которые оказываются в ТОП-150 по правках, составляет 59,8%. Получается, что из числа, кто активно делает правки статей, голосуют лишь чуть больше половины пользователей.  
Посчитано в какой группе (в зависимости от количества раз попадания в ТОП за период 2008-2023гг) люди чаще голосуют и в какой отдают больше всего голосов за одно голосование. Возьмем средние значения по группе.  
Выяснилось, что чем больше раз пользователь попадает в ТОП, тем в большем количестве голосований он участвует и отдает больше голосов, т.е. делает это системно. Больше всего голосов во время одного голосования отдают те, кто попадал в ТОП от 5 до 10 раз, а меньше всего те, кто попадал в ТОП более 75 раз.  
При анализе кандидатов было выявлено, то 37 из них достигали 1 раз необходимого порога 2/3 голосов "за" за время участия в выборах, 24 - 2 раза, 9 - 3раза, 3 - 4 раза, 1 - 6 раз.  
Также построен график, отображающий количество претендентов, которые достигали необходимо порога 2/3 голосов "за" по каждому голосованию (больше всего их было на 9, 12 , 32 голосовании).  

